In [4]:
## import modules
import json
import re
import pandas as pd
import nltk
import math
from math import sqrt
import sklearn
import statsmodels.api as sm
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
from textblob import TextBlob
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer


wnl = WordNetLemmatizer() 
table = str.maketrans('', '', string.punctuation)

In [5]:
## define stopwords
sr = stopwords.words('english')
sr.remove("not")
sr.remove("no")
sr.remove("nor")

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/wanwansu/nltk_data'
    - '/anaconda3/nltk_data'
    - '/anaconda3/share/nltk_data'
    - '/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
## read review data
data_train=pd.read_csv("review_train.csv")
data_train

In [ ]:
## review processiong
review_train_clean = [[1]]*len(data_train)
##a list  with dict element,you can get a summary of frequency of one review by [i] and frequency for a specific word by [i]['word']
frequency_train_clean = [[1]]*len(data_train)  
for i in tqdm(range(len(data_train))):
    #Split into words
    x = word_tokenize(data_train.iloc[i]['text'])
    #Change n't into not
    x = ['not' if w=='n\'t' else w for w in x ]
    #Remove punctuation
    x = [w.translate(table) for w in x]
    #Remove not alphabetic
    x = [word for word in x if word.isalpha()]
    #Convert to lower case
    x = [w.lower() for w in x]
    #Remove stop words
    x = [w for w in x if not w in sr]
    ## lemmatization
    x = [wnl.lemmatize(w) for w in x]
    review_train_clean[i] = x
    frequency_train_clean[i] = collections.Counter(x)

In [ ]:
x = "Worst Gyms I met. And I wouldn't come again."
x=re.sub(r'n\'t', ' not', x, count=0, flags=0)
print(x)

x = word_tokenize(x)
print(x)

    #Remove punctuation
x = [w.translate(table) for w in x]
print(x)

    #Remove not alphabetic
x = [word for word in x if word.isalpha()]
print(x)

    #Convert to lower case
x = [w.lower() for w in x]
print(x)

    #Remove stop words
x = [w for w in x if not w in sr]
print(x)

    ## lemmatization
x = [wnl.lemmatize(w) for w in x]
print(x)


In [ ]:
def flat(l):
    for k in l:
        if not isinstance(k, (list, tuple)):
            yield k
        else:
            yield from flat(k)
allwords = flat(review_train_clean)
allfreq = dict(collections.Counter(allwords))
#Remove words with frequency lower than 10
lowfreqwords=sorted(allfreq.items(), key = lambda item:item[1])
rem_list_low= [lowfreqwords[k][0] for k in range(len(lowfreqwords)) if lowfreqwords[k][1]<5]
#Remove most frequent words
sorted(allfreq.items(), key = lambda item:item[1],reverse=True)[0:10]
#We consider not great like no as important words, so we just add three more useless words
sorted(allfreq.items(), key = lambda item:item[1],reverse=True)[0:14]
#The final remove list for most frequent ones are{gym,class,get,time,one,place,workout,would,go,fitness}
rem_list_high = ['gym','class','get','time','one','place','workout','would','go','fitness']
for i in tqdm(range(len(data_train))):
    x = review_train_clean[i]
    #Remove most frequent words
    x = [w for w in x if not w in rem_list_high]
    #Remove least frequent words
    x = [w for w in x if not w in rem_list_low]
    review_train_clean[i] = x
    frequency_train_clean[i] = collections.Counter(x)

In [3]:
len(review_train_clean)
data_train["words"] = review_train_clean
data_train.to_csv("review_train_aftercleaning.csv",index=False,sep=',')

NameError: name 'review_train_clean' is not defined

In [134]:
data_train_clean = pd.read_csv("review_train_aftercleaning.csv")
data_train_clean

,business_id,date,stars,text,words
0,31292,2013-05-07 04:34:36,1.0,Total bill for this horrible service? Over $8G...,"['total', 'bill', 'horrible', 'service', 'croo..."
1,104374,2015-05-26 07:01:54,4.0,This review is for just the gym facilities as ...,"['review', 'facility', 'never', 'tried', 'tann..."
2,170000,2015-12-28 05:52:45,2.0,There are better and worse Y's for sure. This...,"['better', 'worse', 'sure', 'nothing', 'compar..."
3,51689,2013-12-28 22:45:53,1.0,"Terrible Gym, the ceiling in the mens shower r...","['terrible', 'ceiling', 'men', 'shower', 'room..."
4,69171,2013-04-10 17:15:32,5.0,Fitness together\n\nI can say nothing but posi...,"['together', 'say', 'nothing', 'positive', 'th..."
5,178332,2014-09-25 08:55:35,3.0,This Gold's is a pretty good gym..for the pric...,"['gold', 'pretty', 'good', 'price', 'cheapest'..."
6,51630,2010-08-21 06:53:33,5.0,Wow...no reviews on here yet? I am surprised.....,"['wow', 'no', 'review', 'yet', 'surprised', 'l..."
7,53385,2015-06-08 18:21:10,5.0,I love this place! I started my Bootcamp worko...,"['love', 'started', 'bootcamp', 'week', 'ago',..."
8,151835,2014-10-11 05:55:38,4.0,So far so good...19.99 gets you and a guest i...,"['far', 'good', 'guest', 'staff', 'nice', 'fri..."
9,61726,2018-01-21 14:54:04,3.0,GoodLife member for 3 years now... I personall...,"['goodlife', 'member', 'year', 'personally', '..."


In [5]:
## train words embedding model
## define dimension of need
size = 300
start = time.process_time()
w2v = Word2Vec(review_train_clean, size=size, window=10, min_count=1,
            workers=multiprocessing.cpu_count(), sg=1, iter=10, negative=20)
end = time.process_time()
print('Running time: %s Seconds'%(end-start))
## save model as w2vmodel
w2v.save('w2vmodel')
## load model
model = Word2Vec.load('w2vmodel')

Running time: 810.5755780000001 Seconds


In [137]:
## for each review, get 1*size matrix use average of words matrix 
trysize=5000
predictors = np.empty((trysize,size), float)
for i in tqdm(range(trysize)):
    predictors[i,] = pd.DataFrame(model.wv[review_train_clean[i]]).mean()
    
response = pd.DataFrame(data_train)['stars']


100%|██████████| 5000/5000 [00:01<00:00, 2687.83it/s]


In [180]:
size = 300
trysize=5000
predictors_weighted = np.empty((trysize,size), float)
for i in tqdm(range(trysize)):
    tfidf_dict = tfidf_4_web(review_train_clean[i],data_train)
    tfidf_mul = []
    for word in review_train_clean[i]:
        tfidf_mul.append(tfidf_dict[word])
    predictors_weighted[i,] = (pd.DataFrame(model.wv[review_train_clean[i]]).mul(tfidf_mul,axis=0)).mean()
    
response = pd.DataFrame(data_train)['stars']


100%|██████████| 5000/5000 [11:18<00:00,  9.35it/s]


In [182]:
## logistic regression
model_logit = LogisticRegression(solver = 'lbfgs', multi_class='multinomial',max_iter=500)
model_logit.fit(predictors_weighted, response)
## rmse for regression
y_fitted = model_logit.predict(predictors_weighted)
print("RMSE = ", sqrt(sum(np.square(response-y_fitted)) / len(y_fitted)))

RMSE =  1.331465358167459
